---

# Chapter 22: Refactoring to Patterns

## Opening Context

Knowing design patterns is only half the battle. The real skill lies in recognising when existing code can be improved by applying a pattern and then safely transforming that code without changing its behaviour. This is the art of **refactoring to patterns**.

Refactoring is the process of improving the internal structure of code without altering its external behaviour. Patterns are not just design goals; they are destinations that can be reached through a series of small, behaviour‑preserving steps. When you encounter **code smells**—symptoms of deeper problems—patterns often provide the cure.

This chapter explores three interconnected aspects:

1. **Code Smells as Drivers for Pattern Application** – How to identify smells and map them to appropriate patterns.
2. **Step‑by‑Step Refactoring Techniques** – Safe, incremental moves to introduce patterns.
3. **Test‑Driven Development (TDD) with Patterns** – Using tests to guide the emergence of patterns.

By the end, you will have a practical approach to evolving your code toward cleaner, more maintainable designs.

---

## 22.1 Code Smells as Drivers for Pattern Application

### What Are Code Smells?

Code smells are surface indicators that usually correspond to deeper problems in the code. They are not bugs—they don’t prevent the program from working—but they make the code harder to understand, change, and extend. Recognising smells is the first step toward refactoring.

Common code smells include:
- **Duplicated Code** – The same expression or logic appears in multiple places.
- **Long Method** – A method that is too long and tries to do too many things.
- **Large Class** – A class with too many responsibilities (often a God Object).
- **Long Parameter List** – A method with too many parameters, making it hard to use.
- **Switch Statements / Complex Conditionals** – Frequent type‑checking or conditional logic that could be replaced by polymorphism.
- **Feature Envy** – A method that seems more interested in another class’s data than its own.
- **Inappropriate Intimacy** – Classes that know too much about each other’s internal details.
- **Primitive Obsession** – Overuse of primitive types instead of small objects (e.g., using strings for phone numbers, money, etc.).
- **Speculative Generality** – Code that is designed for future use cases that never materialise.

### Mapping Smells to Patterns

Each smell suggests a family of patterns that can address the underlying problem. Here is a quick reference:

| Smell | Likely Patterns |
|-------|-----------------|
| Duplicated Code | Extract Method, Template Method, Strategy |
| Long Method | Extract Method, Replace Temp with Query, Introduce Parameter Object, Strategy, State |
| Large Class | Extract Class, Extract Subclass, Facade, Proxy |
| Long Parameter List | Introduce Parameter Object, Builder, Factory Method |
| Switch / Complex Conditionals | Replace Conditional with Polymorphism, Strategy, State, Command |
| Feature Envy | Move Method, Move Field, Introduce Foreign Method |
| Inappropriate Intimacy | Change Bidirectional Association to Unidirectional, Extract Class, Hide Delegate |
| Primitive Obsession | Replace Data Value with Object, Introduce Value Object |
| Speculative Generality | Collapse Hierarchy, Inline Class, Remove Parameter |

### Example: Long Method → Strategy Pattern

Consider a method that calculates shipping cost based on the shipping method. It is long and full of conditionals:

```typescript
// ❌ LONG METHOD with conditionals
class Order {
  // ... other properties

  calculateShippingCost(): number {
    let cost = 0;
    if (this.shippingMethod === 'standard') {
      cost = this.totalWeight * 0.5 + 5;
      if (this.totalWeight > 10) {
        cost += 10;
      }
    } else if (this.shippingMethod === 'express') {
      cost = this.totalWeight * 1.5 + 10;
      if (this.totalWeight > 5) {
        cost += 5;
      }
    } else if (this.shippingMethod === 'overnight') {
      cost = this.totalWeight * 3 + 20;
      // no extra
    } else {
      throw new Error('Unknown shipping method');
    }
    return cost;
  }
}
```

**Smell**: The method has multiple responsibilities (logic for each shipping type) and will grow as new methods are added.

**Pattern to apply**: **Strategy**. Encapsulate each shipping algorithm in its own class, and let the `Order` delegate to the chosen strategy.

**Result after refactoring** (shown in Section 22.2).

---

## 22.2 Step‑by‑Step Refactoring Techniques

Refactoring is most effective when done in small, safe steps. Each step should keep the code compiling and tests passing. The process often follows this pattern:

1. **Identify a smell** and decide which pattern might help.
2. **Ensure you have good test coverage** for the area you are about to change.
3. **Introduce the pattern incrementally**, one small step at a time.
4. **Run tests after every change** to verify behaviour is preserved.

Let's walk through a concrete example: refactoring the shipping cost calculation to use the **Strategy** pattern.

### Step 0: Initial Code (with tests)

Assume we have tests for the shipping cost method.

```typescript
// test/order.test.ts
describe('Order shipping cost', () => {
  it('calculates standard shipping', () => {
    const order = new Order({ totalWeight: 8, shippingMethod: 'standard' });
    expect(order.calculateShippingCost()).toBe(8 * 0.5 + 5); // 9
  });
  // ... other tests
});
```

### Step 1: Create Strategy Interface

First, define an interface that represents the shipping cost algorithm.

```typescript
// shipping-strategy.ts
export interface ShippingStrategy {
  calculate(weight: number): number;
}
```

No behaviour change yet; the `Order` still uses the old method.

### Step 2: Extract Each Algorithm into a Concrete Strategy

Create one class for each shipping method, implementing the interface.

```typescript
// strategies/standard-shipping.strategy.ts
export class StandardShippingStrategy implements ShippingStrategy {
  calculate(weight: number): number {
    let cost = weight * 0.5 + 5;
    if (weight > 10) {
      cost += 10;
    }
    return cost;
  }
}

// strategies/express-shipping.strategy.ts
export class ExpressShippingStrategy implements ShippingStrategy {
  calculate(weight: number): number {
    let cost = weight * 1.5 + 10;
    if (weight > 5) {
      cost += 5;
    }
    return cost;
  }
}

// strategies/overnight-shipping.strategy.ts
export class OvernightShippingStrategy implements ShippingStrategy {
  calculate(weight: number): number {
    return weight * 3 + 20;
  }
}
```

### Step 3: Modify Order to Use Strategy via Composition

Add a property to `Order` that holds the current strategy. For now, we'll set it in the constructor or via a setter, but we still keep the old method to avoid breaking existing code.

```typescript
// order.ts (partial)
import { ShippingStrategy } from './shipping-strategy';

export class Order {
  private shippingStrategy: ShippingStrategy;

  constructor(/* ... */, shippingMethod: string) {
    // ... other initializations
    this.setShippingStrategy(shippingMethod);
  }

  setShippingStrategy(method: string): void {
    switch (method) {
      case 'standard':
        this.shippingStrategy = new StandardShippingStrategy();
        break;
      case 'express':
        this.shippingStrategy = new ExpressShippingStrategy();
        break;
      case 'overnight':
        this.shippingStrategy = new OvernightShippingStrategy();
        break;
      default:
        throw new Error('Unknown shipping method');
    }
  }

  // Old method still works
  calculateShippingCost(): number {
    return this.shippingStrategy.calculate(this.totalWeight);
  }
}
```

**Check**: All tests should still pass because the behaviour is identical; we just delegated to the strategy.

### Step 4: Remove the Switch Statement from Order

Now that `Order` uses the strategy, we can remove the method string from the constructor and instead require the client to pass a strategy directly. This is a bigger change; we might do it in two substeps:

- First, add a new constructor that accepts a `ShippingStrategy` and mark the old constructor as deprecated.
- Then, update clients to pass the appropriate strategy.
- Finally, remove the old constructor and the `setShippingStrategy` method.

```typescript
// order.ts (final)
export class Order {
  constructor(
    private totalWeight: number,
    private shippingStrategy: ShippingStrategy
  ) {}

  calculateShippingCost(): number {
    return this.shippingStrategy.calculate(this.totalWeight);
  }
}

// Client code now creates the strategy explicitly
const order = new Order(8, new StandardShippingStrategy());
```

**Benefits**:
- `Order` no longer knows about shipping methods; it just delegates.
- Adding a new shipping method requires creating a new strategy class, not modifying `Order`.
- The shipping algorithms can be tested independently.

### Step‑by‑Step Recap

1. Introduced interface.
2. Extracted algorithms into classes.
3. Added strategy member and used it internally while keeping old method.
4. Updated constructor to accept strategy and removed old conditional.

Each step was small, and tests passed after each change. This is the essence of safe refactoring.

---

## 22.3 Test‑Driven Development (TDD) with Patterns

### TDD Cycle

Test‑Driven Development (TDD) is a technique where you write a failing test first, then write the simplest code to make it pass, and finally refactor to improve the design. Patterns often emerge naturally during the refactoring phase.

### Example: Evolving a Calculator into Interpreter

Suppose we need to evaluate simple arithmetic expressions like "3 + 5" and later "2 * (3 + 4)". We'll use TDD to guide us toward an **Interpreter** pattern.

#### Step 1: Start with the simplest test

```typescript
// test/calculator.test.ts
describe('Calculator', () => {
  it('evaluates a single number', () => {
    const calc = new Calculator();
    expect(calc.evaluate('3')).toBe(3);
  });
});
```

**Implementation** (simplest):

```typescript
class Calculator {
  evaluate(expr: string): number {
    return parseInt(expr, 10);
  }
}
```

#### Step 2: Add test for addition

```typescript
it('adds two numbers', () => {
  const calc = new Calculator();
  expect(calc.evaluate('3+5')).toBe(8);
});
```

Now the simple implementation fails. We need to parse the expression. We could add a quick hack:

```typescript
evaluate(expr: string): number {
  if (expr.includes('+')) {
    const parts = expr.split('+');
    return parseInt(parts[0]) + parseInt(parts[1]);
  }
  return parseInt(expr);
}
```

Test passes. But the code is already getting messy. This is a smell (primitive obsession, long method).

#### Step 3: Add test for subtraction

```typescript
it('subtracts two numbers', () => {
  expect(calc.evaluate('10-3')).toBe(7);
});
```

Now we have to modify the method again, adding more conditionals. This is becoming unmaintainable. Time to refactor.

#### Step 4: Refactor toward a pattern

We notice that each operation (+, -) can be represented as an object. This suggests the **Command** or **Expression** pattern. Let's introduce an interface for expressions.

```typescript
interface Expression {
  interpret(): number;
}
```

Now create classes for numbers and operations:

```typescript
class NumberExpression implements Expression {
  constructor(private value: number) {}
  interpret(): number { return this.value; }
}

class AddExpression implements Expression {
  constructor(private left: Expression, private right: Expression) {}
  interpret(): number { return this.left.interpret() + this.right.interpret(); }
}

class SubtractExpression implements Expression {
  constructor(private left: Expression, private right: Expression) {}
  interpret(): number { return this.left.interpret() - this.right.interpret(); }
}
```

Now we need a parser that builds an abstract syntax tree (AST) from the string. For simplicity, we'll assume the expression is in postfix notation or we'll implement a simple recursive descent parser. This is extra work, but the pattern makes it extensible.

```typescript
class Calculator {
  evaluate(expr: string): number {
    const ast = this.parse(expr);
    return ast.interpret();
  }

  private parse(expr: string): Expression {
    // For demonstration, assume we have a tokenizer and parser.
    // This is simplified; a real parser would handle precedence.
    // But the key is that we return an Expression tree.
    // Example: "3+5" -> new AddExpression(new NumberExpression(3), new NumberExpression(5))
  }
}
```

**Result**: Adding multiplication now just requires a new `MultiplyExpression` class and a small update to the parser—no changes to existing classes. The code follows the **Open/Closed Principle**.

### Patterns Emerge from TDD

Notice that we didn't start with the Interpreter pattern. We started with tests, wrote the simplest code, and when the code became messy, we refactored toward a pattern that solved the recurring problem of adding new operations. This is a common and powerful way to let design emerge organically.

---

## Chapter Summary

Refactoring to patterns is a disciplined way to improve code quality without introducing new bugs. This chapter covered:

1. **Code Smells as Drivers** – Smells like long methods, duplicated code, and complex conditionals point to the need for specific patterns. Recognising these smells is the first step toward better design.

2. **Step‑by‑Step Refactoring Techniques** – Safe refactoring proceeds in tiny, behaviour‑preserving steps. By introducing interfaces, extracting classes, and gradually delegating, you can transform messy code into a pattern‑based solution while keeping tests green.

3. **Test‑Driven Development with Patterns** – TDD encourages simple code first, then refactoring. Patterns often emerge naturally during the refactoring phase, guided by the need to keep tests passing and code clean.

**Key Insight**: Patterns are not destinations you design upfront; they are refactoring targets you reach incrementally. Combined with a solid test suite, this approach yields robust, maintainable code that can evolve with changing requirements.

---

## Next Chapter Preview

**Appendices: Quick Reference Card, Selection Flowchart, Glossary, Recommended Reading**

The next and final section of this handbook provides practical tools: a quick reference cheat sheet of all patterns, a flowchart to help you select the right pattern for a problem, a glossary of key terms, and a curated list of books, articles, and resources for further learning.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. common_anti_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
